In [13]:
%%configure -f
{"executorMemory":"6G", "numExecutors": 4, "driverMemory": "5G",
 "conf":{"spark.executor.cores": 2, "spark.memory.storageFraction": 0.2, "spark.memory.fraction": 0.4,
  "spark.dynamicAllocation.minExecutors": 4, "spark.executor.memoryOverhead": 2000,
  "spark.sql.shuffle.partitions": 20}}

In [ ]:
spark

In [2]:
transactions_df = spark.read.parquet("s3://gigabrain/ethereum_mainnet/transactions/")

In [42]:
logs_df = spark.read.parquet("s3://gigabrain/ethereum_mainnet/logs/")

In [4]:
token_transfers_df = spark.read.parquet("s3://gigabrain/ethereum_mainnet/traces/")

In [5]:
traces_df = spark.read.parquet("s3://gigabrain/ethereum_mainnet/traces/")

22/12/17 00:04:42 WARN org.apache.spark.sql.execution.datasources.SharedInMemoryCache: Evicting cached table partition metadata from memory due to size constraints (spark.sql.hive.filesourcePartitionFileCacheSize = 262144000 bytes). This may impact query planning performance.


In [6]:
"""
All deployed contracts for Uniswap V3 & Uniswap V2
"""
contracts = {'UniswapV3Factory': '0x1f98431c8ad98523631ae4a59f267346ea31f984',
 'Multicall2': '0x5ba1e12693dc8f9c48aad8770482f4739beed696',
 'ProxyAdmin': '0xb753548f6e010e7e680ba186f9ca1bdab2e90cf2',
 'TickLens': '0xbfd8137f7d1516d3ea5ca83523914859ec47f573',
 'Quoter': '0xb27308f9f90d607463bb33ea1bebb41c27ce5ab6',
 'SwapRouter': '0xe592427a0aece92de3edee1f18e0157c05861564',
 'NFTDescriptor': '0x42b24a95702b9986e82d421cc3568932790a48ec',
 'NonfungibleTokenPositionDescriptor': '0x91ae842a5ffd8d12023116943e72a606179294f3',
 'TransparentUpgradeableProxy': '0xee6a57ec80ea46401049e92587e52f5ec1c24785',
 'NonfungiblePositionManager': '0xc36442b4a4522e871399cd717abdd847ab11fe88',
 'V3Migrator': '0xa5644e29708357803b5a882d272c41cc0df92b34',
 'UniswapV2Router02': '0x7a250d5630b4cf539739df2c5dacb4c659f2488d',
 'UniswapV2Factory': '0x5c69bee701ef814a2b6a3edd4b1652cb9cc5aa6f',
 'UniswapV2Router01': '0xf164fc0ec4e93095b804a4795bbe1e041497b92a'}

# Transactions Table

In [8]:
transactions_df = transactions_df.filter("datestr = '2022-12-15'").cache()
transactions_df.count()

22/12/17 00:04:44 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


1080176

In [52]:
"""
The contract specific payloads are decoded into decoded_data column as string serialized JSON
"""
contract = contracts["UniswapV2Router02"]
uniswap_v2_router_transactions = transactions_df.filter(f"to_address = '{contract}'").select("block_number", "block_timestamp", "hash", "from_address", "to_address", "function_name", "contract_name", "decoded_data", "schema")

In [53]:
uniswap_v2_router_transactions.show(50)

+------------+-------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+
|block_number|    block_timestamp|                hash|        from_address|          to_address|       function_name|    contract_name|        decoded_data|              schema|
+------------+-------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+
|    16193419|2022-12-15 23:37:11|0x84388ac74391c2d...|0x402a63f4bcbebd9...|0x7a250d5630b4cf5...|swapExactTokensFo...|UniswapV2Router02|{"amountIn": 1106...|[{"internalType":...|
|    16187051|2022-12-15 02:15:59|0xee57c75cf9eeb49...|0x148797a9339cdbe...|0x7a250d5630b4cf5...|swapExactTokensFo...|UniswapV2Router02|{"amountIn": 6316...|[{"internalType":...|
|    16189924|2022-12-15 11:54:23|0x8d17987d9957437...|0xcea4dc7d171060a...|0x7a250d5630b4cf5...|swapETHF

### Data Munging in Pandas

In [55]:
uniswap_df_12_15 = uniswap_v2_router_transactions.toPandas()

In [56]:
uniswap_df_12_15 = uniswap_df_12_15.to_dict("records")

In [57]:
# Deserialise decoded_data into a dict.
import json
for row in uniswap_df_12_15:
    if row["decoded_data"]:
        row["decoded_data"] = json.loads(row["decoded_data"])

In [58]:
uniswap_df_12_15[:10]

[{'block_number': 16193419,
  'block_timestamp': Timestamp('2022-12-15 23:37:11'),
  'hash': '0x84388ac74391c2dc544d647c713b15114cc89c2231750682f991b92466e05b67',
  'from_address': '0x402a63f4bcbebd9d12e649b2f63573fa40e9dec1',
  'to_address': '0x7a250d5630b4cf539739df2c5dacb4c659f2488d',
  'function_name': 'swapExactTokensForETHSupportingFeeOnTransferTokens',
  'contract_name': 'UniswapV2Router02',
  'decoded_data': {'amountIn': 11067000000000,
   'amountOutMin': 41633272611328635,
   'path': ['0xc1fAD8869D96522090391e7AfFe1eB192A8AFEB4',
    '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'],
   'to': '0x402A63F4bcbEBd9D12E649B2f63573Fa40E9DEC1',
   'deadline': 1671147542},
  'schema': '[{"internalType": "uint256", "name": "amountIn", "type": "uint256"}, {"internalType": "uint256", "name": "amountOutMin", "type": "uint256"}, {"internalType": "address[]", "name": "path", "type": "address[]"}, {"internalType": "address", "name": "to", "type": "address"}, {"internalType": "uint256", "name": "

### PySpark UDFs

In [60]:
import pyspark.sql.functions as F
import pyspark.sql.types as T

# Lets de-normalize decoded_data (note the schema is unique to each function_name)

def to_dict(str_col):
    return json.loads(str_col)

swapExactTokensForETH_schema = T.StructType([
    T.StructField("amountIn",T.StringType(),True),
    T.StructField("path",T.ArrayType(T.StringType()),True),
    T.StructField("to",T.StringType(),True),
    T.StructField("deadline",T.LongType(),True),
  ])
deserialize_swapExactTokensForETH = F.udf(to_dict, swapExactTokensForETH_schema)
swapExactTokensForETH = uniswap_v2_router_transactions.filter("function_name = 'swapExactTokensForETH'").withColumn("decoded", deserialize_swapExactTokensForETH(F.col("decoded_data"))).select("*", "decoded.*").drop("decoded")

In [61]:
swapExactTokensForETH.show(10)

+------------+-------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|block_number|    block_timestamp|                hash|        from_address|          to_address|       function_name|    contract_name|        decoded_data|              schema|            amountIn|                path|                  to|  deadline|
+------------+-------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|    16189833|2022-12-15 11:36:11|0x2c1862da8de949e...|0x07782748dafbc5d...|0x7a250d5630b4cf5...|swapExactTokensFo...|UniswapV2Router02|{"amountIn": 2053...|[{"internalType":...|20534628177276033...|[0x6E954a8f26fBcB...|0x07782748Dafbc5D...|

# Logs Table

In [40]:
logs_df.printSchema()

root
 |-- log_index: long (nullable = true)
 |-- transaction_hash: string (nullable = true)
 |-- transaction_index: long (nullable = true)
 |-- address: string (nullable = true)
 |-- data: string (nullable = true)
 |-- topics: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- block_number: long (nullable = true)
 |-- block_hash: string (nullable = true)
 |-- block_timestamp: timestamp (nullable = true)
 |-- event_name: string (nullable = true)
 |-- contract_name: string (nullable = true)
 |-- decoded_data: string (nullable = true)
 |-- schema: string (nullable = true)
 |-- datestr: string (nullable = true)



In [ ]:
"""
Now we want to collect associated logs for all the transactions we filtered above.
"""

In [64]:
logs_12_15 = logs_df.filter(f"datestr = '2022-12-15'").select("block_number", "transaction_hash", "block_timestamp", "address", "event_name", "contract_name", "decoded_data", "schema")

In [65]:
uniswap_v2_router_logs = uniswap_v2_router_transactions.select("hash").join(logs_12_15, uniswap_v2_router_transactions.hash == logs_12_15.transaction_hash, "left")

In [66]:
uniswap_v2_router_logs.filter("transaction_hash = '0x84388ac74391c2dc544d647c713b15114cc89c2231750682f991b92466e05b67'").show(50)

+--------------------+------------+--------------------+-------------------+--------------------+----------+-------------+--------------------+--------------------+
|                hash|block_number|    transaction_hash|    block_timestamp|             address|event_name|contract_name|        decoded_data|              schema|
+--------------------+------------+--------------------+-------------------+--------------------+----------+-------------+--------------------+--------------------+
|0x84388ac74391c2d...|    16193419|0x84388ac74391c2d...|2022-12-15 23:37:11|0xedb1bb40a6ab502...|      Sync|UniswapV2Pair|{"reserve0": 1590...|[{"indexed": fals...|
|0x84388ac74391c2d...|    16193419|0x84388ac74391c2d...|2022-12-15 23:37:11|0xc1fad8869d96522...|  Transfer|        NoCap|{"from": "0x402A6...|[{"indexed": true...|
|0x84388ac74391c2d...|    16193419|0x84388ac74391c2d...|2022-12-15 23:37:11|0xc02aaa39b223fe8...|  Transfer|        WETH9|{"src": "0xedb1bB...|[{"indexed": true...|
|0x84388ac

In [68]:
uniswap_v2_router_logs.count()

151057

In [69]:
tx_0x84388ac74391c2dc544d647c713b15114cc89c2231750682f991b92466e05b67_logs = uniswap_v2_router_logs.filter("transaction_hash = '0x84388ac74391c2dc544d647c713b15114cc89c2231750682f991b92466e05b67'").toPandas().to_dict("records")

In [71]:
# Deserialise decoded_data into a dict.
import json
for row in tx_0x84388ac74391c2dc544d647c713b15114cc89c2231750682f991b92466e05b67_logs:
    if row["decoded_data"]:
        row["decoded_data"] = json.loads(row["decoded_data"])

tx_0x84388ac74391c2dc544d647c713b15114cc89c2231750682f991b92466e05b67_logs

[{'hash': '0x84388ac74391c2dc544d647c713b15114cc89c2231750682f991b92466e05b67',
  'block_number': 16193419,
  'transaction_hash': '0x84388ac74391c2dc544d647c713b15114cc89c2231750682f991b92466e05b67',
  'block_timestamp': Timestamp('2022-12-15 23:37:11'),
  'address': '0xedb1bb40a6ab5021311647c0abca62eb702a9363',
  'event_name': 'Sync',
  'contract_name': 'UniswapV2Pair',
  'decoded_data': {'reserve0': 1590044449971765032,
   'reserve1': 374554969578382},
  'schema': '[{"indexed": false, "internalType": "uint112", "name": "reserve0", "type": "uint112"}, {"indexed": false, "internalType": "uint112", "name": "reserve1", "type": "uint112"}]'},
 {'hash': '0x84388ac74391c2dc544d647c713b15114cc89c2231750682f991b92466e05b67',
  'block_number': 16193419,
  'transaction_hash': '0x84388ac74391c2dc544d647c713b15114cc89c2231750682f991b92466e05b67',
  'block_timestamp': Timestamp('2022-12-15 23:37:11'),
  'address': '0xc1fad8869d96522090391e7affe1eb192a8afeb4',
  'event_name': 'Transfer',
  'contrac

# Token Transfers

In [26]:
token_transfers_df.select("from_address", "to_address", "block_timestamp", "block_number", "symbol", "name", "value", "decimals").show(100)

+--------------------+--------------------+-------------------+------------+--------------------+--------------------+--------------------+--------+
|        from_address|          to_address|    block_timestamp|block_number|              symbol|                name|               value|decimals|
+--------------------+--------------------+-------------------+------------+--------------------+--------------------+--------------------+--------+
|0xc73e0383f3aff32...|0xad37fd42185ba63...|2020-03-12 00:23:58|     9653504|DAI              ...|Dai Stablecoin v1...| 1817267254389553720|      18|
|0xf75fcd9849d3365...|0x794e6e91555438a...|2020-03-12 00:23:58|     9653504|                 DAI|      Dai Stablecoin|20945100000000000...|      18|
|0x794e6e91555438a...|0xf28b5298066658c...|2020-03-12 02:15:52|     9654016|                 DAI|      Dai Stablecoin|67826325347693500...|      18|
|0x2bee010be3b17f0...|0x3360b113c0228c3...|2020-03-12 03:09:57|     9654272|MKR              ...|Maker    